In [0]:
from pyspark.sql.functions import current_timestamp, current_date, lit, col, max as spark_max

def add_bronze_metadata(df, source_table, load_type):
    return (
        df
        .withColumn("_source_table", lit(source_table))
        .withColumn("_ingestion_timestamp", current_timestamp())
        .withColumn("_ingestion_date", current_date())
        .withColumn("_load_type", lit(load_type))
    )


def get_last_incremental_value(target_table, incremental_column):
    if spark.catalog.tableExists(target_table):
        return (
            spark.table(target_table)
            .select(spark_max(col(incremental_column)))
            .collect()[0][0]
        )
    return None


def write_bronze_table(
    df,
    target_table,
    load_type,
    incremental_column=None,
    partition_column=None,
    optimize=False,
    vacuum=True,
    vacuum_retention_hours=168  # 7 days (default Delta safe retention)
):
    writer = (
        df.write
        .format("delta")
        .mode("append" if load_type == "INCREMENTAL" else "overwrite")
    )

    if partition_column:
        writer = writer.partitionBy(partition_column)

    writer.saveAsTable(target_table)

    if optimize:
        spark.sql(f"OPTIMIZE {target_table}")


    if vacuum:
        spark.sql(
            f"VACUUM {target_table} RETAIN {vacuum_retention_hours} HOURS"
        )
    
    print(f"Sucess: {target_table}")
